In [1]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import pytz

# Get IST current hour
india_time = datetime.now(pytz.timezone('Asia/Kolkata'))
current_hour = india_time.hour

if 18 <= current_hour < 21:
    # --- Filter dataset ---
    df = user_review.copy()
    
    # Convert date column if not already
    df['date'] = pd.to_datetime(df['date'])

    # Apply filters
    df = df[
        (~df['app'].str.lower().str.startswith(('x', 'y', 'z'))) &
        (df['category'].str.upper().str.startswith(('E', 'C', 'B'))) &
        (df['reviews'] > 500)
    ]

    # Add year-month for grouping
    df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

    # Aggregate installs per month and category
    monthly_installs = df.groupby(['month', 'category'])['installs'].sum().reset_index()

    # Compute % change
    monthly_installs['pct_change'] = monthly_installs.groupby('category')['installs'].pct_change()

    # --- Streamlit Slider for Date Filter ---
    min_date = monthly_installs['month'].min()
    max_date = monthly_installs['month'].max()
    
    start_date, end_date = st.slider(
        "Select Date Range",
        min_value=min_date,
        max_value=max_date,
        value=(min_date, max_date),
        format="YYYY-MM"
    )

    # Filter data based on selected date range
    monthly_installs_filtered = monthly_installs[
        (monthly_installs['month'] >= start_date) & 
        (monthly_installs['month'] <= end_date)
    ]

    # --- Plotting with Plotly ---
    fig = go.Figure()

    categories = monthly_installs_filtered['category'].unique()

    for category in categories:
        sub = monthly_installs_filtered[monthly_installs_filtered['category'] == category]
        
        # Plot line for installs
        fig.add_trace(go.Scatter(
            x=sub['month'],
            y=sub['installs'],
            mode='lines+markers',
            name=category,
            text=sub['installs'],
            hoverinfo='x+y+text',  # Show month and installs in tooltip
            line=dict(width=2)
        ))

        # Highlight periods of >20% MoM growth
        growth_periods = sub[sub['pct_change'] > 0.2]
        fig.add_trace(go.Scatter(
            x=growth_periods['month'],
            y=growth_periods['installs'],
            mode='markers',
            name=f'{category} >20% Growth',
            marker=dict(size=10, color='rgba(255, 165, 0, 0.7)'),  # Orange color for growth periods
            text=growth_periods['installs'],
            hoverinfo='x+y+text',
            showlegend=False
        ))

    # Update layout
    fig.update_layout(
        title="Monthly Install Trend by Category (Highlighted >20% MoM Growth)",
        xaxis_title="Month",
        yaxis_title="Total Installs",
        hovermode="x unified",  # Display tooltips when hovering on a specific month across all lines
        template="plotly_dark",  # Dark theme
        xaxis=dict(tickformat="%b %Y"),  # Format x-axis as Month Year
    )

    # Show Plotly figure in Streamlit
    st.plotly_chart(fig)

else:
    st.warning("This chart is only visible from 6 PM to 9 PM IST.")


2025-05-06 22:48:36.670 
  command:

    streamlit run C:\Users\Raghuraman\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
